In [1]:
import sys
import os

# append the path of the current notebook to the sys.path
cur_dir = os.path.abspath(os.path.curdir)
root_dir = os.path.abspath(os.path.join(cur_dir, '..'))
sys.path.append(root_dir)

In [2]:
import pandas as pd

In [3]:
from src.data.load_data import load_data
from src.features.engineering import engineer_features
from src.models.evaluate import evaluate_model
from src.data.split_data import split_data
from sklearn.ensemble import RandomForestClassifier
from src.models.RandomForest import train_RandomForest
from src.features.selection import select_features

In [16]:
data = engineer_features(load_data(), 'readmitted', one_hot=True)
col = data.select_dtypes(include='uint8').columns

--------------------
Feature engineering complete.
--------------------


/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:51: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data_encoded = pd.get_dummies(data.drop(columns=[target]), columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:51: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data_encoded = pd.get_dummies(data.drop(columns=[target]), columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:51: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data_encoded = pd.get_dummies(data.drop(columns=[target]), columns=cat_cols, drop_first=True)


In [20]:
data.astype('float64').dtypes

time_in_hospital      float64
num_lab_procedures    float64
num_procedures        float64
num_medications       float64
number_outpatient     float64
                       ...   
A1Cresult_None        float64
A1Cresult_Norm        float64
change_No             float64
diabetesMed_Yes       float64
readmitted            float64
Length: 503, dtype: object

In [6]:
data.shape

(101766, 47)

In [24]:
# decision tree model
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [16]:
X = data.drop('readmitted', axis=1)
y = data['readmitted']

# Extract categorical columns (replace 'categorical_columns' with the actual names of your categorical columns)
# Replace with your categorical column names
X_categorical = X.select_dtypes(include=['object'])
cat_cols = X_categorical.columns

# Initialize the OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Fit and transform the categorical columns
X_encoded = ordinal_encoder.fit_transform(X_categorical)

# Replace the categorical columns in X with the encoded values
X.loc[:, cat_cols] = X_encoded

/var/folders/t6/rx8j2pqs09l9rckj9vd_4zt80000gn/T/ipykernel_51057/479915836.py:16: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[:, cat_cols] = X_encoded


In [21]:
X = data.drop('readmitted', axis=1)
y = data['readmitted']

# one hot encoding
# Initialize the OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

# Fit and transform the categorical columns
X_encoded = one_hot_encoder.fit_transform(X)

/Users/dumindu/anaconda3/envs/conda-diabetes/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [29]:
data_enc = data.apply(LabelEncoder().fit_transform)

In [35]:
data_enc

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2,0,0,5,23,0,0,7,37,40,...,0,1,1,0,0,0,0,1,0,2
1,2,0,1,0,0,6,2,7,18,58,...,0,3,1,0,0,0,0,0,1,1
2,0,0,2,0,0,6,1,7,18,10,...,0,1,1,0,0,0,0,1,1,2
3,2,1,3,0,0,6,1,7,18,43,...,0,3,1,0,0,0,0,0,1,2
4,2,1,4,0,0,6,0,7,18,50,...,0,2,1,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,0,1,7,0,2,6,2,7,18,50,...,0,0,1,0,0,0,0,0,1,1
101762,0,0,8,0,3,4,4,7,18,32,...,0,2,1,0,0,0,0,1,1,2
101763,2,1,7,0,0,6,0,7,18,52,...,0,0,1,0,0,0,0,0,1,2
101764,2,0,8,1,2,6,9,7,62,44,...,0,3,1,0,0,0,0,0,1,2


In [11]:
dtree = DecisionTreeClassifier()

dtree.fit(data.drop('readmitted', axis=1), data['readmitted'])

ValueError: could not convert string to float: 'Caucasian'

In [12]:
data.dtypes

race                         object
gender                       object
age                          object
admission_type_id            object
discharge_disposition_id     object
admission_source_id          object
time_in_hospital            float64
payer_code                   object
medical_specialty            object
num_lab_procedures          float64
num_procedures              float64
num_medications             float64
number_outpatient           float64
number_emergency            float64
number_inpatient            float64
diag_1                       object
diag_2                       object
diag_3                       object
number_diagnoses            float64
max_glu_serum                object
A1Cresult                    object
metformin                    object
repaglinide                  object
nateglinide                  object
chlorpropamide               object
glimepiride                  object
acetohexamide                object
glipizide                   

In [38]:
x = 1